Setup the imports

In [ ]:
import subprocess
import sys


def install_package(package: str):
    subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
    

def download_spacy_model(model: str):
    subprocess.check_call([sys.executable, '-m', 'spacy', 'download', model])

In [1]:
import json
import random
import collections
import dataclasses
from typing import *

In [2]:
import numpy as np

import nltk
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords

from npdl.utils import one_hot

In [3]:
import npdl
from npdl import activations
from npdl import layers
from npdl import objectives
from npdl import optimizers

## Load the data

In [4]:
with open('intents.json', 'r') as file:
    intents = json.load(file)

## Preprocess the data

In [5]:
Token = str


@dataclasses.dataclass
class Document:
    class_: str
    tokens: Iterable[Token]

def tokenize(sentence: str,
             language: str = 'english') -> Iterable[Token]:
    stop_words = ['?'] + stopwords.words(language)
    return np.array([SnowballStemmer(language).stem(word.lower()) for word in nltk.word_tokenize(sentence, language)
                     if word not in stop_words and word.isalpha()])

Initialize the **vocabulary**, the **classes** and the **corpus**.

In [6]:
vocabulary: List[Token] = []
corpus: List[Document] = []
classes: List[str] = []

In [7]:
for document in intents:
    class_ = document['class']
    classes.append(class_)
    for pattern in document['patterns']:
        tokens = tokenize(pattern)
        vocabulary.extend(tokens)
        corpus.append(Document(class_, tokens))

In [8]:
vocabulary = np.sort(np.unique(vocabulary))
classes = np.sort(np.unique(classes))

## Setup the training data

In [9]:
def one_hot_encoder(train_labels, labels):
    label_positions = {}
    for i, label in enumerate(train_labels):
        label_positions[label] = i
    
    encoded = np.zeros((len(labels), len(train_labels)))
    for i, label in enumerate(labels):
        j = label_positions[label]
        encoded[i, j] = 1
    return encoded

def frenquency_encoder(train_labels, labels):
    label_frequencies = {}
    for label in train_labels:
        label_frequencies[label] = label_frequencies.get(label, 0) + 1
    
    return [label_frequencies.get(label, 0) for label in labels]

In [10]:
train_X = np.array([frenquency_encoder(document.tokens, vocabulary) for document in corpus])
train_Y = one_hot_encoder(classes, np.array([document.class_ for document in corpus]))

## Setup the model

In [11]:
model = npdl.Model()

shape = (train_X.shape[1], 64, train_Y.shape[1])

model.add(layers.Dense(n_in=shape[0], n_out=500, activation=activations.ReLU()))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(shape[1], activation=activations.ReLU()))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(shape[2], activation=activations.Softmax()))

In [12]:
model.compile(loss=objectives.SCCE(), optimizer=optimizers.Adam())

## Train the model

In [13]:
model.fit(train_X, train_Y, batch_size=5)

iter 1, train-[loss 1.3845, acc 0.3000]; 
iter 2, train-[loss 1.3867, acc 0.1000]; 
iter 3, train-[loss 1.3797, acc 0.3000]; 
iter 4, train-[loss 1.3506, acc 0.4000]; 
iter 5, train-[loss 1.3101, acc 0.7000]; 
iter 6, train-[loss 1.2676, acc 0.5000]; 
iter 7, train-[loss 1.3276, acc 0.4000]; 
iter 8, train-[loss 1.2525, acc 0.6000]; 
iter 9, train-[loss 1.2294, acc 0.7000]; 
iter 10, train-[loss 1.2140, acc 0.6000]; 
iter 11, train-[loss 1.1810, acc 0.9000]; 
iter 12, train-[loss 1.1208, acc 0.8000]; 
iter 13, train-[loss 1.1151, acc 0.8000]; 
iter 14, train-[loss 1.0991, acc 0.9000]; 
iter 15, train-[loss 1.1145, acc 0.8000]; 
iter 16, train-[loss 0.9561, acc 0.9000]; 
iter 17, train-[loss 1.1352, acc 0.7000]; 
iter 18, train-[loss 1.0403, acc 0.8000]; 
iter 19, train-[loss 0.9698, acc 0.8000]; 
iter 20, train-[loss 0.8863, acc 1.0000]; 
iter 21, train-[loss 0.9684, acc 0.9000]; 
iter 22, train-[loss 0.8763, acc 1.0000]; 
iter 23, train-[loss 0.8315, acc 1.0000]; 
iter 24, train-[loss

In [14]:
def predict(query):
    tokens = tokenize(query)
    vector = frenquency_encoder(tokens, vocabulary)
    prediction = model.predict([vector])
    index = prediction.argmax()
    return classes[index]

def answer(query):
    predicted_class = predict(query)
    answers = next(filter(lambda intent: intent['class'] == predicted_class, intents))['responses']
    return random.choice(answers)